# Discussion 3: Line Search Algorithms, Wolfe Conditions

In this discussion, we will talk about:
* Choice of step direction $\mathbf{p}_k$ in line search algorithms
* Choice of step size $\alpha_k$ in steepest descent
* Wolfe conditions
    

---

## Line search algorithms

Recall that the basic idea behind iterative optimization algorithms is to create a sequence $\mathbf{x}_1$, $\mathbf{x}_2$, $\ldots$, $\mathbf{x}_k$, $\ldots$ which satisfies $f(\mathbf{x}_1)\ge f(\mathbf{x}_2)\ge\cdots\ge f(\mathbf{x}_k)\ge\cdots$, i.e. that the successive iterates better and better approximate the minimum value of the function $f$. The common thread connecting all **line search algorithms** is to choose at each iteration $\mathbf{x}_k$ some direction $\mathbf{p}_k$ in which to "step", and set

$$ \mathbf{x}_{k+1}=\mathbf{x}_k+\alpha_k\mathbf{p}_k $$

for some positive scalar value $\alpha_k$ called the "step length", determined at each iteration. The choice of $\mathbf{p}_k$ determines the algorithm.

### Determining the step direction

A Taylor expansion of the function $f$ near the current iterate $\mathbf{x}_k$ to second order gives

$$ f(\mathbf{x}_{k+1})=f(\mathbf{x}_k+\alpha_k\mathbf{p}_k) = f(\mathbf{x}_k) + \alpha_k\mathbf{p}_k^T\nabla f(\mathbf{x}_k) + \frac{\alpha_k^2}{2}\mathbf{p}_k^T\nabla^2f(\mathbf{x}_k+t\mathbf{p}_k)\mathbf{p}_k$$

for some $t\in(0,\alpha_k)$. Thus, even if $\nabla^2f$ is positive definite, if $\alpha_k$ is sufficiently small, the linear term dominates, and we can guarantee $f_{k+1}\equiv f(\mathbf{x}_{k+1})\le f_k\equiv f(\mathbf{x}_k)$ if $\mathbf{p}_k$ is chosen such that $\mathbf{p}_k^T\nabla f_k<0$, i.e. if $\mathbf{p}_k$ forms an angle of larger than $\pi/2$ with the gradient $\nabla f_k$. Any direction that satisfies this condition is called a **descent direction**.

The direction of *steepest* descent is simply $\mathbf{p}_k=-\nabla f_k$, and indeed this choice of step direction defines the **steepest descent** (aka **gradient descent** or simply "GD") method,
$$ \mathbf{x}_{k+1} = \mathbf{x}_k - \alpha_k\nabla f_k $$

This is the simplest choice of step direction and is therefore the most commonly used and most theoretically explored, but there are other methods which use different descent directions.

For example, **gradient descent with momentum** is defined by

$$ \begin{align*}
    \mathbf{v}_k &= \alpha_k\nabla f_k \color{red}{+ \gamma \mathbf{v}_{k-1}} \\
    \mathbf{x}_{k+1} &= \mathbf{x}_k - \mathbf{v}_k
\end{align*} $$

where $\gamma\in[0,1]$ is a parameter which determines what fraction of the previous update vector $\mathbf{v}_{k-1}$ we add into the current update vector $\mathbf{v}_k$, and $\mathbf{v}_0\equiv0$. In some sense, we can think of this as adding *momentum* to the updates since if two consecutive updates are in basically the same direction, this algorithm will move more quickly in that direction to try to reach the minimum faster. If the previous update was in a direction opposite to the current one, it has the effect of slowing down the descent.

Another method is **Nesterov's accelerated gradient** (NAG), which is defined by

$$ \begin{align*}
    \mathbf{v}_k &= \alpha_k\nabla f(\color{red}{\mathbf{x}_k-\gamma\mathbf{v}_{k-1}})+ \gamma \mathbf{v}_{k-1} \\
    \mathbf{x}_{k+1} &= \mathbf{x}_k - \mathbf{v}_k
\end{align*} $$

which is essentially the same as adding momentum, but modifying where the gradient is calculated, in a kind of "look ahead" manner.

Yet another method which involves information about the Hessian of $f$ (a *second-order* method) is **Newton's method**, which defines

$$ \mathbf{x}_{k+1}=\mathbf{x}_k-\alpha_k\left(\nabla^2f_k\right)^{−1}\nabla f_k $$

which requires $\nabla^2f_k$ to be nonsingular (or even positive definite) in order to compute the inverse. We will discuss this method in more detail later in the quarter.

In each of the above, one can show that the choice of step direction (e.g. $\mathbf{p}_k=-\nabla f_k-\frac{\gamma}{\alpha_k}\mathbf{v}_{k-1}$ in GD with momentum) is indeed a descent direction, possibly under certain conditions on $\alpha_k$.

### Determining the step length

Given some choice of step direction $\mathbf{p}_k$, we must also determine how far we want to move in that direction, which is in general a more difficult problem. If we want to choose an *optimal* value of $\alpha_k$, we could set
$$ \alpha_k = \min_{\alpha>0} f(\mathbf{x}_k+\alpha \mathbf{p}_k) $$

i.e., look infinitely far along the line in the direction of $\mathbf{p}_k$ and find the smallest value of $f$ in that direction. To find the value of $\alpha$ that minimizes the RHS, we could think of it as a (one-dimensional!) function of $\alpha$, say, $\phi(\alpha)=f(\mathbf{x}+\alpha \mathbf{p}_k)$ and find where $\phi'(\alpha)=0$. This method of determining the step length is called the **exact line search** algorithm, and can be combined with any choice of descent direction. For simplicity, in the examples below, we choose $\mathbf{p}_k=-\nabla f_k$, and thus use the **steepest descent exact line search** method.

#### Example 1: An analytic solution

Consider the function $f(x,y) = (x-1)^2+ (y−2)^2$. For any point $\mathbf{x}_k=\langle x_k,y_k\rangle$, determine the optimal value of $\alpha_k$ along the direction $\mathbf{p}_k=-\nabla f_k$.

**Solution**: Since
$$ \mathbf{p}_k=-\nabla f_k = \langle -2(x_k-1), -2(y_k-2)\rangle $$
we have
$$ \begin{align*}
    \alpha_k &= \min_{\alpha>0} f(\mathbf{x}_k+\alpha \mathbf{p}_k) = \min_{\alpha>0} f\Big(\langle x_k,y_k\rangle-\alpha\langle2(x_k-1),2(y_k-2)\rangle\Big) \\
    &= \min_{\alpha>0} f\Big(\langle x_k-2\alpha(x_k-1),y_k-2\alpha(y_k-2)\rangle\Big) \\
    &= \min_{\alpha>0} \big(x_k-2\alpha(x_k-1) - 1\big)^2 + \big(y_k-2\alpha(y_k-2)-2\big)^2 \\
    &= \min_{\alpha>0} (1-2\alpha)^2\Big[(x_k-1)^2+(y_k-2)^2\Big]
\end{align*}$$

In this case, we get some nice factorization and the optimal value of $\alpha$ is obviously $\alpha=1/2$, when the factor out front equals zero. Thus, no matter the starting point, the optimal value is $\alpha_k=1/2$, a constant step size.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# define the function
f = lambda x,y: (x-1)**2 + (y-2)**2
# define derivatives of f to make the gradient
Df = lambda x,y: np.array([2*(x-1), 2*(y-2)])

# plot the figure first
plt.figure(figsize=(6, 6))
X = np.linspace(0,5,300)  # 300 evenly spaced points on x-axis [0,5]
Y = np.linspace(0,5,300)  # 300 evenly spaced points on y-axis [0,5]
Xmesh, Ymesh = np.meshgrid(X,Y)  # 300x300 grid of points defined by X and Y above
Z = f(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, 20, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([0,5,0,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.random.rand(2)*5  # initial point randomly chosen
print(f'Initial x={x}')
dx = np.array([np.inf,np.inf]) # initial large gradient so while loop runs
alpha = 0.5           # optimal step size
tol = 1e-3            # stop when gradient is smaller than this amount
max_steps = 100       # Maximum number of steps to run the iteration
i=0                   # iteration count
while np.linalg.norm(dx)>tol and i<max_steps:
    dx = Df(x[0],x[1])
    # new value of x
    xnew = x - alpha*dx
    # add arrow to plot
    plt.arrow(x[0],x[1],-alpha*dx[0],-alpha*dx[1],color='b',head_width=.1)
    # update old value
    x = xnew
    # update iteration count
    i += 1
    print(f'In iteration {i}, alpha={alpha}, and newx={x}')

print(f'After {i} iterations, approximate minimum is {f(x[0],x[1])} at {x}')    
plt.show()

#### Example 2: Slightly more difficult

Repeat the above for the function $g(x,y) = (x-1)^2+ (2y−1)^2$.

**Solution**: Since
$$ \mathbf{p}_k=-\nabla g_k = \langle -2(x_k-1), -4(2y_k-1)\rangle $$
we have
$$ \begin{align*}
    \alpha_k &= \min_{\alpha>0} f(\mathbf{x}_k+\alpha \mathbf{p}_k) \\
    &= \min_{\alpha>0} f\Big(\langle x_k-2\alpha(x_k-1),y_k-4\alpha(2y_k-1)\rangle\Big) \\
    &= \min_{\alpha>0} \big(x_k-2\alpha(x_k-1) - 1\big)^2 + \big(2y_k-8\alpha(2y_k-1)-1\big)^2 \\
    &= \min_{\alpha>0} (1-2\alpha)^2(x_k-1)^2+(1-8\alpha)^2(2y_k-1)^2
\end{align*}$$

In this case, the optimal value of $\alpha$ is not immediately obvious, so we resort to single variable calculus. Setting

$$ \phi(\alpha) = (1-2\alpha)^2(x_k-1)^2+(1-8\alpha)^2(2y_k-1)^2$$
we have
$$ 0=\phi'(\alpha) = -4(x_k-1)^2(1-2\alpha) - 16(2y_k-1)^2(1-8\alpha) 
    = 4\bigg[\alpha\Big(2(x_k-1)^2+32(2y_k-1)^2\Big) - \Big((x_k-1)^2+4(2y_k-1)^2\Big)\bigg]$$

$$ \implies \alpha=\frac{(x_k-1)^2+4(2y_k-1)^2}{2(x_k-1)^2+32(2y_k-1)^2}$$

Thus, the optimal value for $\alpha_k$ depends on the current location, e.g. if we are at $\mathbf{x}_k=\langle 3,3\rangle$, the optimal value of $\alpha_k=104/808\approx0.1287$.

In [1]:
alpha = lambda x,y: ((x-1)**2+4*(2*y-1)**2)/(2*(x-1)**2+32*(2*y-1)**2)
alpha(3,3)

0.12871287128712872

In [2]:
x,y=np.random.rand()*5,np.random.rand()*5
print(f'At {x},{y}\nalpha={alpha(x,y)}') # random point in [0,5]x[0,5]

NameError: name 'np' is not defined

In [ ]:
# define the function
g = lambda x,y: (x-1)**2 + (2*y-1)**2
# define derivatives of f to make the gradient
Dg = lambda x,y: np.array([2*(x-1), 4*(2*y-1)])

# plot the figure first
plt.figure(figsize=(6, 6))
X = np.linspace(0,5,300)  # 300 evenly spaced points on x-axis [0,5]
Y = np.linspace(0,5,300)  # 300 evenly spaced points on y-axis [0,5]
Xmesh, Ymesh = np.meshgrid(X,Y)  # 300x300 grid of points defined by X and Y above
Z = g(Xmesh,Ymesh)
CS = plt.contour(Xmesh, Ymesh, Z, 20, cmap='jet')
plt.clabel(CS,inline_spacing=0,fmt='%d')
plt.axis([0,5,0,5])
plt.xlabel('x')
plt.ylabel('y')

x = np.random.rand(2)*5  # initial point randomly chosen
print(f'Initial x={x}')
dx = np.array([np.inf,np.inf]) # initial large gradient so while loop runs
tol = 1e-3            # stop when gradient is smaller than this amount
max_steps = 100       # Maximum number of steps to run the iteration
i=0                   # iteration count
while np.linalg.norm(dx)>tol and i<max_steps:
    dx = Dg(x[0],x[1])
    # new value of x
    xnew = x - alpha(x[0],x[1])*dx # note alpha is a function here!
    # add arrow to plot
    plt.arrow(x[0],x[1],-alpha(x[0],x[1])*dx[0],-alpha(x[0],x[1])*dx[1],color='b',
                      head_width=.1,length_includes_head=True)
    # update old value
    x = xnew
    # update iteration count
    i += 1
    print(f'In iteration {i}, alpha={alpha(x[0],x[1])}, and newx={x}')

print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')    
plt.show()

### Wolfe conditions

Now of course we probably don't want to sit down and do all these calculations out by hand each time we want to minimize a new function, and we likely simply *can't* do this for very high-dimensional functions, so instead of always looking for the *best* $\alpha_k$, we often settle for a "*good enough*" $\alpha_k$. But what exactly does "good enough" mean? Enter the **Wolfe conditions**:

* Wolfe I (Armijo condition): $\qquad f_{k+1}\le f_k+c_1\alpha_k\mathbf{p}_k^T\nabla f_k$
* Wolfe II (curvature condition): $\quad \mathbf{p}_k^T\nabla f_{k+1}\ge c_2\mathbf{p}_k^T\nabla f_k$

where $0<c_1<c_2<1$. Given a choice of $c_1$ and $c_2$, it can be shown analytically that the iteration will converge, i.e. that $\nabla f_k\to0$ as $k\to\infty$.

#### Wolfe I

The first of these conditions in essence gives an upper limit on the value that $\alpha_k$ can take, and the second gives a lower limit. To see this, realize that the LHS of Wolfe I is just $\phi(\alpha)=f(\mathbf{x}_k+\alpha\mathbf{p}_k)$, and so we can plot it for a given $f$, $\mathbf{x}_k$, and $\mathbf{p}_k$, as below for the function $g(x,y)=(x-1)^2+ (2y−1)^2$ we explored above:

In [ ]:
xk = np.array([3,3])   # Initial point (3,3)
pk = -Dg(xk[0],xk[1])  # step direction
# phi(a) = g(x + a*p) = g(x - a*dg)
phi = lambda alpha: g(xk[0]+alpha*pk[0],  xk[1]+alpha*pk[1])
xx = np.linspace(0,0.5,100)
pp = phi(xx)
plt.figure(dpi=100)
plt.plot(xx,pp)
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\phi(\alpha)$')
plt.ylim([0,50])
plt.title(f'At x={xk}, p={-Dg(xk[0],xk[1])}')
plt.show()

Indeed we see the optimal value of $\alpha$ on this plot is $\alpha\approx 0.1287$ as we determined above with the exact line search.

Now the RHS of Wolfe I is linear in $\alpha$, and indeed is equal to $\phi(0)+\alpha c_1\phi'(0)$. Since we have chosen $\mathbf{p}_k$ such that $\phi'(0)<0$, the line has a negative slope for $c_1\in(0,1)$. Indeed if $c_1=0$ this is just a horizontal line with value $f_k=\phi(0)$ and if $c_1=1$, the slope is $\mathbf{p}_k^T\nabla f_k=\phi'(0)$, as shown below:

In [ ]:
plt.figure(dpi=100)
plt.plot(xx,pp,label=r'$\phi(\alpha)$')
for i in range(9):
    ll = phi(0)-(1/2**i)*xx*np.dot(pk,pk)
    plt.plot(xx,ll,':',label=f'c_1={1/2**i}')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\phi(\alpha)$')
plt.ylim([0,50])
plt.title(f'At x={xk}, p={-Dg(xk[0],xk[1])}')
plt.legend(loc=(1.1,.25))
plt.show()

So all the Wolfe I condition is saying is we should choose $\alpha$ such that the function value is smaller than that of the line with slope defined by $c_1$, effectively setting an upper bound on $\alpha$. Given a $c_1$, we can solve for the value of $\alpha$ where the line intersects $\phi(\alpha)$ and choose any value smaller than that. In practice, $c_1$ is usually chosen very small ($\sim10^{-4}$) to provide sufficient convergence. For our discussion, let's choose $c_1=1/8$, the purple line above, which looks like it intersects $\phi(\alpha)$ at around $\alpha\approx 0.22$.

#### Wolfe II

While the Wolfe I condition sets an *upper* bound on $\alpha$, it does not set a lower bound. This means that although we can guarantee the function decreases sufficiently, we cannot guarantee the function decreases sufficiently *quickly*. This is where the Wolfe II condition, $\mathbf{p}_k^T\nabla f_{k+1}\ge c_2\mathbf{p}_k^T\nabla f_k$, comes in. Note that the LHS is simply $\phi'(\alpha_k)$ and the RHS is $c_2\phi'(0)$. Thus, Wolfe II says that we should choose $\alpha$ such that the slope of $\phi$ at the new point is *less steep* than the current gradient. This is true since $c_2<1$. For example, if $c_2=0.5$ (the green line in our plot above), we exclude $\alpha$ very close to zero since $\phi$ is very steep there, and determine the lower bound to be where the tangent line of $\phi$ has slope $c_2\phi'(0)$ (looks to be around $\alpha\approx 0.05$).

### Example: Analytically calculating Wolfe condition limits

Continuing with our function $g$ above, we now set $c_1=1/8$ and $c_2=0.5$ and determine the range of allowable $\alpha$. We have already calculated

$$\phi(\alpha) = g_{k+1} = (1-2\alpha)^2(x_k-1)^2+(1-8\alpha)^2(2y_k-1)^2$$

which when $\mathbf{x}_k=\langle 3,3\rangle$ gives

$$g_{k+1}= 4(1-2\alpha)^2+25(1-8\alpha)^2$$

Similarly, $g_k=g(3,3)=29$ and $\mathbf{p}_k=-\nabla g_k=\langle-4,-20\rangle$. Then Wolfe I requires

$$ \begin{align*}
    g_{k+1} &\le g_k+c_1\alpha\mathbf{p}_k^T\nabla g_k \\
    \implies 4(1-2\alpha)^2+25(1-8\alpha)^2 &\le 29+\frac{1}{8}\alpha\cdot-416 \\
    \implies 1616\alpha^2 - 364\alpha &\le 0 \\
    \implies \alpha &\le \frac{364}{1616}\approx0.2252
\end{align*}$$
which matches the upper limit we expect on the graph above.

Now Wolfe II requires computing

$$\mathbf{p}_k^T\nabla g_{k+1}=\phi'(\alpha)=-4(x_k-1)^2(1-2\alpha) - 16(2y_k-1)^2(1-8\alpha)=3232\alpha-416$$
Then
$$ \begin{align*}
    \mathbf{p}_k^T\nabla g_{k+1} &\ge c_2\mathbf{p}_k^T\nabla g_k \\
    \implies 3232\alpha-416 &\ge \frac{1}{2}\cdot -416 \\
    \implies \alpha\ge \frac{208}{3232}\approx0.0644
\end{align*} $$
which again matches the lower limit we expect from the graph above.

Putting the above together, when $\mathbf{x}_k=\langle 3,3\rangle$, the range of acceptable values of $\alpha_k\in(0.0644, 0.2252)$.

### Conclusion

After performing the calculations above, a decent guess for the optimal value of $\alpha$ might be the midpoint of the interval, $\alpha\approx 0.1448$. This is indeed very close to the calculated optimal value of $\alpha=0.1287$.

In reality, though, even calculating the bounds for the Wolfe conditions is difficult for general high-dimensional nonlinear functions, so we may need to relax our requirements a bit to determine a better heuristic for determining $\alpha$ that is easier to calculate but still has good convergence properties. That is what we will talk about next time.